# Problem Set 7

### Dan Gamarnik

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import LeaveOneOut, KFold

C:\Users\GamaD\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
auto = pd.read_csv('Auto.csv', na_values='?')
auto = auto.dropna()

In [3]:
auto['mpg_median'] = auto['mpg'].median()
auto['mpg_high'] = auto['mpg'] > auto['mpg_median']
auto['mpg_high'] = auto['mpg_high'].apply(int)
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,mpg_median,mpg_high
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,22.75,0
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,22.75,0
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,22.75,0
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,22.75,0
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,22.75,0


In [4]:
yvals = auto['mpg_high'].values
Xvals = auto[['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']].values
k = 4
kf = KFold(n_splits=k, random_state=15, shuffle=True)
kf.get_n_splits(Xvals)
Logit_err_0 = np.zeros(k)
Logit_err_1 = np.zeros(k)
Logit_MSE_VEC = np.zeros(k)

In [5]:
k_ind = int(0)
for train_index, test_index in kf.split(Xvals):
    # print("TRAIN:", train_index, "TEST:", test_index)
    print('If k index=', k_ind + 1)
    X_train, X_test = Xvals[train_index], Xvals[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    LogReg = LogisticRegression(fit_intercept=True)
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    err = y_test != y_pred
    Logit_MSE_VEC[k_ind] = err.mean()
    Logit_err_0[k_ind] = ((y_pred == 0) * err).sum() / (y_pred == 0).sum() 
    Logit_err_1[k_ind] = ((y_pred == 1) * err).sum() / (y_pred == 1).sum() 
    print(classification_report(y_test, y_pred))
    print('The error rate(category 0) is', Logit_err_0[k_ind])
    print('The error rate(category 1) is', Logit_err_1[k_ind])
    print('The MSE is', Logit_MSE_VEC[k_ind])
    k_ind += 1
print('\n k-Fold Reults:')
print('The average error rate(category 0) is', Logit_err_0.mean())
print('The average error rate(category 1) is', Logit_err_1.mean())
print('The average MSE of the model is', Logit_MSE_VEC.mean())

If k index= 1
             precision    recall  f1-score   support

          0       0.94      0.89      0.92        55
          1       0.87      0.93      0.90        43

avg / total       0.91      0.91      0.91        98

The error rate(category 0) is 0.0576923076923
The error rate(category 1) is 0.130434782609
The MSE is 0.0918367346939
If k index= 2
             precision    recall  f1-score   support

          0       0.88      0.91      0.90        47
          1       0.92      0.88      0.90        51

avg / total       0.90      0.90      0.90        98

The error rate(category 0) is 0.122448979592
The error rate(category 1) is 0.0816326530612
The MSE is 0.102040816327
If k index= 3
             precision    recall  f1-score   support

          0       0.85      0.87      0.86        45
          1       0.88      0.87      0.88        53

avg / total       0.87      0.87      0.87        98

The error rate(category 0) is 0.152173913043
The error rate(category 1) is 0.1

In [6]:
from sklearn.ensemble import RandomForestClassifier
Xvals = auto[['cylinders','displacement','horsepower','weight','acceleration','year','origin']].values
rf = RandomForestClassifier(n_estimators=20, max_features=2, bootstrap=True,
                           oob_score=True, random_state=25)
rf.fit(Xvals, yvals)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=True, random_state=25, verbose=0, warm_start=False)

In [7]:
rf_pred = rf.oob_decision_function_
MSE_rf = pd.DataFrame({'ypred' : rf_pred.T[1], 'yvals': yvals})
MSE_rf['ypred'] = MSE_rf['ypred'].apply(lambda x: 1 if x >= 0.5 else 0)
MSE_model = mean_squared_error(MSE_rf['yvals'], MSE_rf['ypred'])
print('The MSE of the random forest model is:',MSE_model)
MSE_rf_0 = MSE_rf[MSE_rf['ypred'] < 0.5]
MSE_0 = mean_squared_error(MSE_rf_0['yvals'], MSE_rf_0['ypred'])
print('The MSE of the mpg_high = 0 is:',MSE_0)
MSE_rf_1 = MSE_rf[MSE_rf['ypred'] >= 0.5]
MSE_1 = mean_squared_error(MSE_rf_1['yvals'], MSE_rf_1['ypred'])
print('The MSE of the mpg_high = 1 is:',MSE_1)

The MSE of the random forest model is: 0.0714285714286
The MSE of the mpg_high = 0 is: 0.0578947368421
The MSE of the mpg_high = 1 is: 0.0841584158416


In [8]:
# c). 
from sklearn.svm import SVC
error_0 = np.zeros(k)
error_1 = np.zeros(k)
MSE_vec_kf = np.zeros(k)
k_ind = int(0)
for train_index, test_index in kf.split(Xvals):
    # print("TRAIN:", train_index, "TEST:", test_index)
    print('k index=', k_ind)
    X_train, X_test = Xvals[train_index], Xvals[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    svc = SVC(kernel='rbf', gamma = 0.2, C=1)
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    MSE_vec_kf[k_ind] = ((y_test - y_pred) ** 2).mean()
    e_0 = ((y_pred==0)*(np.abs(y_test-y_pred))).sum() / (y_pred==0).sum()
    error_0[k_ind] = (e_0)
    e_1 = ((y_pred==1)*(np.abs(y_test-y_pred))).sum() / (y_pred==1).sum()
    error_1[k_ind] = (e_1)
    print('MSE for test set', k_ind, ' is', MSE_vec_kf[k_ind])
    print('Error rate (low, 0): ', e_0)
    print('Error rate (high, 1): ', e_1)
    print('\n',classification_report(y_test, y_pred, digits=3))
    k_ind += 1

MSE_kf = MSE_vec_kf.mean()
MSE_kf_std = MSE_vec_kf.std()
print()
print('test estimate MSE k-fold=', MSE_kf)

error_0 = error_0[~np.isnan(error_0)]
error_1 = error_1[~np.isnan(error_1)]
print('The average error rate of mpg_high = 0 is', error_0.mean())
print('The average error rate of mpg_high = 1 is', error_1.mean())

k index= 0
MSE for test set 0  is 0.540816326531
Error rate (low, 0):  0.0
Error rate (high, 1):  0.552083333333

              precision    recall  f1-score   support

          0      1.000     0.036     0.070        55
          1      0.448     1.000     0.619        43

avg / total      0.758     0.459     0.311        98

k index= 1
MSE for test set 1  is 0.520408163265
Error rate (low, 0):  0.520408163265
Error rate (high, 1):  nan

              precision    recall  f1-score   support

          0      0.480     1.000     0.648        47
          1      0.000     0.000     0.000        51

avg / total      0.230     0.480     0.311        98

k index= 2
MSE for test set 2  is 0.520408163265
Error rate (low, 0):  0.53125
Error rate (high, 1): 

C:\Users\GamaD\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\GamaD\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


 0.0

              precision    recall  f1-score   support

          0      0.469     1.000     0.638        45
          1      1.000     0.038     0.073        53

avg / total      0.756     0.480     0.332        98

k index= 3
MSE for test set 3  is 0.448979591837
Error rate (low, 0):  0.47311827957
Error rate (high, 1):  0.0

              precision    recall  f1-score   support

          0      0.527     1.000     0.690        49
          1      1.000     0.102     0.185        49

avg / total      0.763     0.551     0.438        98


test estimate MSE k-fold= 0.507653061224
The average error rate of mpg_high = 0 is 0.381194110709
The average error rate of mpg_high = 1 is 0.184027777778


Based on the above results it is clear that the the random forest is the best model. This is because the error rates 
for this model are the lowest. The MSE is also lowest on this model. The SVM is the least accurate of the three and the 
logistic regression model is the next best after the random forest. Notice that MSE on the SVM is approximately 0.507 while on the logit model it is about 0.107 and on the random forest it is roughly 0.071. This trend also continues in errorrates in which the random forest has the lowest high and low mpg error rates.
